In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from gensim.models import Word2Vec
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_books_final = pd.read_csv('df_books_final.csv')

combine test and training
for each user id inside, recommend some books,
compare that recommendation with what he actually read 

### using df_books_final_with_desc as main dataset

In [3]:
df_books_final_with_desc = pd.read_csv('df_books_final_with_desc.csv')

In [4]:
df_books_final_with_desc_sampled = df_books_final_with_desc.sample(frac = 0.7)

In [5]:
df_books_final_with_desc_sampled = df_books_final_with_desc_sampled.reset_index()

In [6]:
print(df_books_final.shape)
print(df_books_final_with_desc_sampled.shape)

(85466, 106)
(59826, 108)


In [7]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_scores =  tfidf.fit_transform(df_books_final_with_desc_sampled['description'])
tfidf_scores.shape

(59826, 162088)

In [8]:
# cosine similarity matrix directly for the sparse TF-IDF matrix
tfidf_cos_sim = cosine_similarity(tfidf_scores, dense_output=False)

In [9]:
df_books_final_with_desc_sampled.head(5)

,index,book_id,description,average_rating,ratings_count,text_reviews_count,top_popular_shelves,author_ids,title_tfidf,description_word2vec,...,lang_tr,lang_tur,lang_ukr,lang_vi,lang_vie,lang_yid,lang_zh,length_long,length_medium,length_short
0,82472,542266,The Second Book In The Award-Winning Whalesong...,0.475815,-0.045564,-0.090230,"[{'count': '4', 'name': 'childrens'}, {'count'...",['3004'],"(0, 30165)\t0.4951721287250606\n (0, 6653)\...",[ 1.239543 0.373717 0.07649366 0.108885...,...,False,False,False,False,False,False,False,False,True,False
1,19740,31213715,"Are your toes getting sleepy? So very, very sl...",-0.386863,-0.049182,-0.106269,"[{'count': '3', 'name': 'board-books'}, {'coun...","['5254942', '2851649']","(0, 19777)\t0.5918897536992717\n (0, 25046)...",[ 1.9632746 0.52620447 0.25226402 0.069296...,...,False,False,False,False,False,False,False,False,False,True
2,48123,13093858,"Pa morgonen nar mamma vacker honom, sager den ...",0.225360,-0.039964,-0.074191,"[{'count': '7', 'name': 'picture-book'}, {'cou...","['232839', '424553']","(0, 3892)\t0.7251025224756457\n (0, 21256)\...",[ 4.3354958e-02 6.8408020e-02 7.7701017e-02 ...,...,False,False,False,False,False,False,False,False,False,True
3,81673,28592980,"Meet Mr. Fox, who loves books so much that eve...",-1.416511,-0.049182,-0.078201,"[{'count': '6', 'name': 'picture-books'}, {'co...",['580168'],"(0, 1704)\t0.6614687081774673\n (0, 9783)\t...",[ 2.24585915e+00 6.87894940e-01 3.01568300e-...,...,False,False,False,False,False,False,False,False,False,True
4,17201,13252019,"During the days of the Great War, Captain Amer...",0.225360,-0.047976,-0.102259,"[{'count': '2', 'name': 'childrens'}, {'count'...","['4908941', '10294']","(0, 1854)\t0.5168052118717642\n (0, 13698)\...",[ 1.8278023 0.4543071 0.45529813 0.046496...,...,False,False,False,False,False,False,False,False,False,True


In [28]:
def recommend_books(books_df, target_book_id, similarity_matrix, top_n=5):
    # check if the target_book_id exists in the books_df
    if target_book_id not in books_df['book_id'].values:
        # print(f"Book ID {target_book_id} not found in books_df")
        return []  # Return an empty list if the book_id isn't found
    
    # find index of referenced book
    target_index = books_df.index[books_df['book_id'] == target_book_id].tolist()[0]

    # get similarity scores for the target book
    similarity_scores = list(enumerate(similarity_matrix[target_index].toarray().flatten()))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # get the indices of the top_n most similar books, excluding the target book itself
    similar_indices = [i for i, score in similarity_scores[1:top_n+1]]
    
    # get details of the recommended books
    recommended_books = books_df.iloc[similar_indices]
    
    return recommended_books['book_id'].tolist()

In [29]:
# example test case
recommendations = recommend_books(df_books_final_with_desc_sampled, target_book_id=542266, similarity_matrix=tfidf_cos_sim, top_n=5)
print(recommendations)

[281889, 3436918, 30403004, 14992931, 14992929]


# recommendation on training set

In [32]:
train_interactions = pd.read_csv('train_interactions.csv')
# test_interactions = pd.read_csv('test_interactions.csv')
# val_interactions = pd.read_csv('val_interactions.csv')

,user_id,book_id,rating,n_votes,review_age,sentiment
0,1,39343,2,0.0,3434,0.851100
1,1,21648,0,NaN,3435,0.406046
2,2,7794,4,0.0,3046,-0.421500
3,3,8637,3,NaN,3343,0.406860
4,3,50719,4,NaN,3367,0.527973


In [56]:
# Check if any book_id in train_interactions exists in df_books_final_with_desc_sampled
exists_in_df = train_interactions['book_id'].isin(df_books_final_with_desc_sampled['book_id']).any()
print(f"Do any book_ids from train_interactions exist in df_books_final_with_desc_sampled? {exists_in_df}")

Do any book_ids from train_interactions exist in df_books_final_with_desc_sampled? True


In [61]:
# # checks

# # Get the list of common book_ids between train_interactions and df_books_final_with_desc_sampled
# common_book_ids = train_interactions[train_interactions['book_id'].isin(df_books_final_with_desc_sampled['book_id'])]['book_id'].unique()
# # print("Common book_ids:", common_book_ids)

# # Check if book_id 39343 exists in df_books_final_with_desc_sampled
# book_id_to_check = 3839
# exists_in_df = book_id_to_check in df_books_final_with_desc_sampled['book_id'].values
# print(f"Does book_id {book_id_to_check} exist in df_books_final_with_desc_sampled? {exists_in_df}")

In [59]:
# filter for those train_users who have read books inside df_books_final_with_desc_sampled
filtered_train_interactions = train_interactions[train_interactions['book_id'].isin(df_books_final_with_desc_sampled['book_id'])]

In [67]:
# aggregate recommendations per user
from collections import Counter

# Aggregate recommendations for each user
def get_aggregated_recommendations(user_books, books_df, similarity_matrix, top_n=1):
    recommended_books = []
    
    # Collect recommendations for each book a user has read
    for book_id in user_books:
        book_recommendations = recommend_books(books_df, book_id, similarity_matrix, top_n)
        recommended_books.extend(book_recommendations)
    
    # Count recommendations to rank by frequency
    recommendation_counts = Counter(recommended_books)
    
    # Filter out books already read and get the most frequently recommended
    recommended_books = [
        book_id for book_id, _ in recommendation_counts.most_common() 
        if book_id not in user_books
    ]
    
    return recommended_books[:top_n]

# Group by user_id to get all books a user has read
user_recommendations = {}
for user_id, user_df in filtered_train_interactions.groupby('user_id'):
    user_books = user_df['book_id'].tolist()
    user_recommendations[user_id] = get_aggregated_recommendations(user_books, df_books_final_with_desc_sampled, tfidf_cos_sim, top_n=1)

# add user recommendations to train_interactions DataFrame by mapping
filtered_train_interactions['user_recommendations'] = filtered_train_interactions['user_id'].map(user_recommendations)
print(filtered_train_interactions.head(3))

     user_id  book_id  rating  n_votes  review_age  sentiment  \
107       20     3839       5      NaN        6002   0.574139   
423       45    38644       5      NaN        2997   0.574139   
488       45    33627       4      NaN        3295   0.527973   

    user_recommendations  
107               [9440]  
423           [12137531]  
488           [12137531]  


/var/folders/lv/4p2_jtmx6rzdf0lvp89413tc0000gn/T/ipykernel_25036/339418261.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_interactions['user_recommendations'] = filtered_train_interactions['user_id'].map(user_recommendations)


test

In [62]:
test_interactions = pd.read_csv('test_interactions.csv')
test_interactions.head(3)

,user_id,book_id,rating,n_votes,review_age,sentiment
0,1,8882,0,NaN,3497,0.406046
1,2,9402,5,0.0,3047,-0.598200
2,3,945,5,NaN,3705,0.574139


In [71]:
# filter for those train_users who have read books inside df_books_final_with_desc_sampled
filtered_test_interactions = test_interactions[test_interactions['book_id'].isin(df_books_final_with_desc_sampled['book_id'])]

In [73]:
# Get unique user IDs from filtered_train_interactions
train_user_ids = filtered_train_interactions['user_id'].unique()
filtered_test_interactions = filtered_test_interactions[filtered_test_interactions['user_id'].isin(train_user_ids)]

In [75]:
filtered_test_interactions.head(3)

,user_id,book_id,rating,n_votes,review_age,sentiment
257,56,3839,0,NaN,5300,0.406046
580,118,5209,5,NaN,4606,0.574139
776,148,42714,5,NaN,2782,0.574139


In [89]:
# similarity_threshold = 0.5

# evaluation_results = []

# # For each user, compare the similarity between recommended and actual book
# for user_id, user_df in filtered_train_interactions.groupby('user_id'):
#     # Get the recommended book_id for the user (assuming only 1 recommendation)
#     recommended_book_id = user_df['user_recommendations'].iloc[0][0]  # since it's a list
    
#     # Get the actual book_id that the user read in the test set
#     actual_book_id = filtered_test_interactions[filtered_test_interactions['user_id'] == user_id]['book_id'].iloc[0]
    
#     # Get the indices of the books in the similarity matrix
#     recommended_index = df_books_final_with_desc_sampled[df_books_final_with_desc_sampled['book_id'] == recommended_book_id].index[0]
#     actual_index = df_books_final_with_desc_sampled[df_books_final_with_desc_sampled['book_id'] == actual_book_id].index[0]
    
#     # Get the similarity score between the recommended and actual book
#     similarity_score = tfidf_cos_sim[recommended_index, actual_index]
    
#     # Check if the similarity score is above the threshold
#     if similarity_score >= similarity_threshold:
#         evaluation_results.append((user_id, recommended_book_id, actual_book_id, similarity_score, True))
#     else:
#         evaluation_results.append((user_id, recommended_book_id, actual_book_id, similarity_score, False))

# # Convert the results to a DataFrame for easier analysis
# evaluation_df = pd.DataFrame(evaluation_results, columns=['user_id', 'recommended_book_id', 'actual_book_id', 'similarity_score', 'correct'])

# # Calculate the accuracy (proportion of correct recommendations)
# accuracy = evaluation_df['correct'].mean()
# print(f"Recommendation accuracy: {accuracy:.2f}")

In [84]:
# Define a function to calculate similarity score between two book_ids
def calculate_similarity(book_id_1, book_id_2, books_df, similarity_matrix):
    try:
        index_1 = books_df.index[books_df['book_id'] == book_id_1].tolist()[0]
        index_2 = books_df.index[books_df['book_id'] == book_id_2].tolist()[0]
        similarity_score = similarity_matrix[index_1, index_2]
        return similarity_score
    except IndexError:
        return None  # none if the book_id doesnt exist in the similarity matrix

similarity_threshold = 0.15 #threshold for evaluation
results = []

for _, row in filtered_test_interactions.iterrows():
    user_id = row['user_id']
    actual_book_id = row['book_id']

    # Get the recommended book_id from your user_recommendations dictionary (assuming 1 recommendation per user)
    recommended_books = user_recommendations.get(user_id, [])
    
    # check if recommendations exist for the user
    if recommended_books:
        recommended_book_id = recommended_books[0]  # take the first recommendation
        
        # similarity between actual and recommended book
        similarity = calculate_similarity(actual_book_id, recommended_book_id, df_books_final_with_desc_sampled, tfidf_cos_sim)
        
        # store the results 
        if similarity is not None:
            success = 1 if similarity >= similarity_threshold else 0
            results.append({
                'user_id': user_id,
                'actual_book_id': actual_book_id,
                'recommended_book_id': recommended_book_id,
                'similarity': similarity,
                'success': success
            })
        else:
            results.append({
                'user_id': user_id,
                'actual_book_id': actual_book_id,
                'recommended_book_id': recommended_book_id,
                'similarity': None,
                'success': 0
            })

evaluation_df = pd.DataFrame(results)
success_rate = evaluation_df['success'].mean()
print(f"Recommendation Success Rate: {success_rate * 100:.2f}%")
print(evaluation_df.head())


Recommendation Success Rate: 0.57%
   user_id  actual_book_id  recommended_book_id  similarity  success
0     56.0          3839.0              2162078    0.006569        0
1    118.0          5209.0               632539    0.005373        0
2    148.0         42714.0              7632080    0.008114        0
3    223.0         24184.0               525298    0.040627        0
4    240.0         46308.0                48562    0.000000        0


In [83]:
evaluation_df_sorted = evaluation_df.sort_values(by='similarity', ascending=False)
print(evaluation_df_sorted.head())

     user_id  actual_book_id  recommended_book_id  similarity  success
120   3530.0          5986.0              3975711    0.292900        0
364  11190.0          3004.0              2995910    0.249330        0
149   4317.0         44210.0                48953    0.176757        0
159   4780.0          3003.0             18669257    0.126377        0
383  11446.0         44210.0             10697948    0.112153        0


In [88]:
evaluation_df_sorted.shape

(525, 5)

### Ignore
0 recommendations read

In [64]:
# Create a dictionary to store books read by each user in the test set
test_books_read = test_interactions.groupby('user_id')['book_id'].apply(set).to_dict()
# Initialize a dictionary to store the count of recommendations read for each user
recommendations_match_count = {}
test_books_read

# Loop through each user in filtered_train_interactions
for idx, row in filtered_train_interactions.iterrows():
    user_id = row['user_id']
    recommended_books = row['user_recommendations']

    # Check if the user exists in the test set
    if user_id in test_books_read:
        # Get the set of books read by the user in the test set
        read_books = test_books_read[user_id]
        
        # Count the number of recommended books that were actually read by the user
        match_count = len(set(recommended_books) & read_books)
        
        # Store the result in the dictionary
        recommendations_match_count[user_id] = match_count

# Display the number of matches for each user
for user, count in recommendations_match_count.items():
    print(f"User {user} has {count} recommendation(s) that they actually read in the test set.")

### IGNORE THE CODES BELOW

In [ ]:
# prof code

def cos_sim(vector_a, vector_b):
    # Calculate the dot product of the two vectors
    dot_product = np.dot(vector_a, vector_b)

    # Calculate the Euclidean norm (magnitude) of each vector
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)

    # Calculate cosine similarity using the dot product and vector norms
    similarity = dot_product / (norm_a * norm_b)
    return similarity


def pairwise_cos_sim(vectors):
    num_vectors = vectors.shape[0]
    similarity_matrix = np.zeros((num_vectors, num_vectors))

    for i in range(num_vectors):
        for j in range(num_vectors):
            # Calculate cosine similarity for each pair of vectors
            similarity_matrix[i, j] = cos_sim(vectors[i], vectors[j])

    return similarity_matrix

tfidf_cos_sim = pairwise_cos_sim(tfidf_scores.toarray())
print(tfidf_cos_sim.shape)

# Apply this function to train_interactions
user_recommendations = {}

for user_id, user_df in train_terence.groupby('user_id'):
    user_books = user_df['book_id'].tolist()
    aggregated_recommendations = []
    
    for book_id in user_books:
        book_recommendations = recommend_books(
            df_books_final_with_desc_sampled, 
            book_id, 
            tfidf_cos_sim, 
            top_n=3
        )
        aggregated_recommendations.extend(book_recommendations)
    
    # Store recommendations in a dictionary for each user
    user_recommendations[user_id] = aggregated_recommendations

# Map recommendations to the original dataframe
train_terence['user_recommendations'] = train_terence['user_id'].map(user_recommendations)

In [ ]:
# Function to convert string to NumPy array
def convert_to_vector(vector_str):
    # Remove square brackets and convert string to NumPy array
    vector_str = vector_str.strip('[]')
    vector_list = np.fromstring(vector_str, sep=' ')
    return vector_list

# apply the function to the entire column and create a new column with the vectors
df_books_final['description_word2vec'] = df_books_final['description_word2vec'].apply(convert_to_vector)
# print(df_books_final[['description_word2vec']])

# merge description_word2vec col to training df
merged_train_with_desc = train_interactions.merge(
    df_books_final[['book_id', 'description_word2vec']],
    on='book_id',
    how='left'
)

merged_train_with_desc.head(3) # also have NA values in description
merged_train_with_desc['description_word2vec'] = merged_train_with_desc['description_word2vec'].fillna('')

merged_train_with_desc.head(3)

def cos_sim(vector_a, vector_b):
    # Calculate the dot product of the two vectors
    dot_product = np.dot(vector_a, vector_b)

    # Calculate the Euclidean norm (magnitude) of each vector
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)

    # Calculate cosine similarity using the dot product and vector norms
    similarity = dot_product / (norm_a * norm_b)
    return similarity

def pairwise_cos_sim(vectors):
    num_vectors = vectors.shape[0]
    similarity_matrix = np.zeros((num_vectors, num_vectors))

    for i in range(num_vectors):
        for j in range(num_vectors):
            # Calculate cosine similarity for each pair of vectors
            similarity_matrix[i, j] = cos_sim(vectors[i], vectors[j])

    return similarity_matrix


# Create a TfidfVectorizer object with English stop words (0.1 marks)
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the 'Description' column of the DataFrame to convert the raw
# text in 'Description' to TF-IDF features (0.1 marks)
tfidf_scores =  tfidf.fit_transform(merged_train_with_desc['description_word2vec'])

# Check the shape of the resulting TF-IDF matrix (0.2 marks)
# Row 285: number of documents
# Column 2363: number of features (terms in the vocabulary)
tfidf_scores.shape

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming vector_list1 and vector_list2 are already obtained from the code you provided
# Check if the vectors have the same length
if len(vector_list1) == len(vector_list2):
    # Reshape vectors to be 2D arrays for cosine_similarity
    vector_list1 = vector_list1.reshape(1, -1)
    vector_list2 = vector_list2.reshape(1, -1)

    # Compute cosine similarity using sklearn
    similarity = cosine_similarity(vector_list1, vector_list2)

    # Print the cosine similarity result
    print(f"Cosine similarity between the two descriptions: {similarity[0][0]}")
else:
    print("Vectors have different lengths, cosine similarity cannot be computed.")


In [ ]:
# # fill the empty rows of 'description' column with empty string so tf-idf can be applied
# sampled_merged['description'] = sampled_merged['description'].fillna('')

# # Remove duplicate rows based on the 'book_id' column
# sampled_merged = sampled_merged.drop_duplicates(subset='book_id').reset_index(drop=True)

In [ ]:
top_x_books = sampled_merged.nlargest(10000, 'sentiment') #limit to 10k rows top

In [ ]:
# tfidf = TfidfVectorizer(stop_words='english')
# tfidf_scores =  tfidf.fit_transform(top_x_books['description'])
# tfidf_scores.shape

In [ ]:
# cosine similarity on the reduced dataset
tfidf_cos_sim = cosine_similarity(tfidf_scores, dense_output=False)
print(tfidf_cos_sim.shape) 

In [ ]:
top_x_books.head()

In [ ]:
# # getting top n similar books for a given book
# def get_top_n_similar_books(book_index, similarity_matrix, n=3):
#     # Extract row of similarities for the book of interest
#     sim_scores = similarity_matrix[book_index].toarray().flatten()
    
#     # Get indices of top N scores, excluding the book itself
#     top_indices = sim_scores.argsort()[-n-1:-1][::-1]
    
#     # Return a list of (book index, similarity score)
#     return [(idx, sim_scores[idx]) for idx in top_indices]

# # Example usage: Find top 5 similar books for the book at index 0
# similar_books = get_top_n_similar_books(0, tfidf_cos_sim, n=3)
# print(similar_books)

In [ ]:
# Function to get the top N similar books using the book title
def get_top_n_similar_books_by_title(book_title, similarity_matrix, df, n=3):
    # Find the index of the book title in the dataframe
    book_index = df.index[df['title'] == book_title].tolist()
    
    # If the book title is not found, return an empty list
    if not book_index:
        print(f"Book title '{book_title}' not found in the dataset.")
        return []
    
    book_index = book_index[0]
    
    # Extract row of similarities for the book of interest
    sim_scores = similarity_matrix[book_index].toarray().flatten()
    
    # Get indices of top N scores, excluding the book itself
    top_indices = sim_scores.argsort()[-n-1:-1][::-1]
    
    # Retrieve book_id, title, and similarity score for each top similar book
    top_books = [(df.iloc[idx]['book_id'], df.iloc[idx]['title'], sim_scores[idx]) for idx in top_indices]
    
    return top_books

# Example usage: Find top 3 similar books for a given book title
book_title = sampled_merged.iloc[0]['title']  # Replace with the actual title you're interested in
similar_books = get_top_n_similar_books_by_title(book_title, tfidf_cos_sim, sampled_merged, n=3)

# Print results with book_id, title, and similarity score
for similar_book in similar_books:
    print(f"Book ID: {similar_book[0]}, Title: {similar_book[1]}, Similarity Score: {similar_book[2]:.4f}")


In [ ]:
# tfidf = TfidfVectorizer(stop_words='english')
# tfidf_scores =  tfidf.fit_transform(sampled_merged['description'])
# tfidf_scores.shape

In [ ]:
# def cos_sim(vector_a, vector_b):
#     # Calculate the dot product of the two vectors
#     dot_product = np.dot(vector_a, vector_b)

#     # Calculate the Euclidean norm (magnitude) of each vector
#     norm_a = np.linalg.norm(vector_a)
#     norm_b = np.linalg.norm(vector_b)

#     # Calculate cosine similarity using the dot product and vector norms
#     similarity = dot_product / (norm_a * norm_b)
#     return similarity



# def pairwise_cos_sim(vectors):
#     num_vectors = vectors.shape[0]
#     similarity_matrix = np.zeros((num_vectors, num_vectors))

#     for i in range(num_vectors):
#         for j in range(num_vectors):
#             # Calculate cosine similarity for each pair of vectors
#             similarity_matrix[i, j] = cos_sim(vectors[i], vectors[j])

#     return similarity_matrix




In [ ]:
# sampled_interactions_df : user_id, book_id, sentiment
# sampled_interactions_df does not have book name and description
# map sampled_interactions_df book id to df_children_books_final to get title, description
# new sampled_interactions_df should have: user_id, book_id, sentiment

# 1) create tfidfvectoriser
# 2) fit_transform the vectoriser on sampled_interactions_df['title']
# 3) define cos_sim() function to get cosine similarity value of 2 vectors
# 4) define pairwise_cos_sim() that makes use of cos_sim()
# 5) apply pairwise_cos_sim() function to tfidf_scores.to_array()
# 6) define recommend_books() function that takes in sampling_interaction_df ,title (for recommendation reference), tfidf_cos_sim matrix, # number of similar recs)
# 7) test with a random book title -> Dog Heaven